<a href="https://colab.research.google.com/github/yukinaga/ai_novel/blob/main/section_3/02_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-2のファインチューニング
青空文庫の小説を訓練データに使い、GPT-2のモデルをファインチューニングします。  
ファインチューニングにより、モデルから漱石風の文章が生成されるようになることを確認しましょう。   
学習に時間がかかるので、「編集」→「ノートブックの設定」の「ハードウェアアクセラレーター」で「GPU」を選択しましょう。   

## ライブラリのインストール
GPT-2が含まれるライブラリtransformers、形態素解析（≒単語分割）のためのライブラリsentencepieceをインストールします。

In [ ]:
!pip install transformers
!pip install sentencepiece

## GPT-2の設定

今回は`rinna/japanese-gpt2-medium `をベースとして使用します。   

In [ ]:
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

## 訓練データの読み込みと前処理
以下のレポジトリをダウンロードして解凍し、section_3の「wagahaiwa_nekodearu.txt」をアップロードしましょう。  
https://github.com/yukinaga/ai_novel  
  
以下のコードを実行すると、ローカル環境からファイルをアップロードできます。  

In [ ]:
from google.colab import files

uploaded = files.upload()  # ファイルのアップロード
file_path_original = list(uploaded.keys())[0]  # ファイルパス
print(file_path_original)  

アップロードされたファイルを読み込み、一部を表示します。  

In [ ]:
with open(file_path_original, mode="r", encoding="utf-8") as f:  # ファイルの読み込み
    text_original = f.read()

print(text_original[:100])  # 最初の100文字を表示

正規表現を使い、ルビなどを削除します。

In [ ]:
import re

text = re.sub("《[^》]+》", "", text_original)  # ルビの削除
text = re.sub("［[^］]+］", "", text)  # 読みの注意の削除
text = re.sub("[｜ 　]", "", text)  # | と全角半角スペースの削除
print(text[:100])  # 最初の100文字を表示

train_data_path = "train.txt"
with open(train_data_path, mode="w") as f:
    f.write(text)

## モデルの訓練
既存のモデルに追加で訓練を行います。  
まずは、訓練のための各設定を行います。  

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

# データセットの設定
train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_data_path,
        block_size=128  # 文章の長さ
        )

# データの入力に関する設定
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # データをマスクするかどうか
    )

# 訓練に関する設定
training_args = TrainingArguments(
    output_dir="./gpt2-ft",  # 関連ファイルを保存するパス
    overwrite_output_dir=True,  # ファイルを上書きするかどうか
    num_train_epochs=3,  # エポック数
    per_device_train_batch_size=8,  # バッチサイズ
    logging_steps=100,  # 途中経過を表示する間隔
    save_steps=800  # モデルを保存する間隔
    )

# トレーナーの設定
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

トレーナーの`train()`メソッドにより、訓練が開始されます。 

In [ ]:
trainer.train()

## 文章を生成する関数
入力文章から続きの文章を生成する関数を設定します。  


In [ ]:
def getarate_sentences(seed_sentence):
    x = tokenizer.encode(seed_sentence, return_tensors="pt", add_special_tokens=False)  # 入力
    x = x.cuda()  # GPU対応
    y = model.generate(x, #　入力
                       min_length=50,  # 文章の最小長
                       max_length=100,  # 文章の最大長
                       do_sample=True,   # 次の単語を確率で選ぶ
                       top_k=50, # Top-Kサンプリング
                       top_p=0.95,  # Top-pサンプリング
                       temperature=1.2,  # 確率分布の調整
                       num_return_sequences=3,  # 生成する文章の数
                       pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
                       bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
                       eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
                       bad_word_ids=[[tokenizer.unk_token_id]]  # 生成が許可されないトークンID
                       )  
    generated_sentences = tokenizer.batch_decode(y, skip_special_tokens=True)  # 特殊トークンをスキップして文章に変換
    return generated_sentences

## 文章の生成
「吾輩は猫である」の冒頭をシードにして、ファインチューニング済みのGPT-2モデルにより小説を執筆します。

In [ ]:
seed_sentence = "吾輩は猫である。名前はまだ無い。"  # 吾輩は猫であるの冒頭
generated_sentences = getarate_sentences(seed_sentence)  # 生成された文章
for sentence in generated_sentences:
    print(sentence)

シードの文章にアレンジを加えましょう。

In [ ]:
seed_sentence = "吾輩は犬である。名前は"  # シード文章
generated_sentences = getarate_sentences(seed_sentence)  # 生成された文章
for sentence in generated_sentences:
    print(sentence)

## 文章の保存
生成した文章を、txtファイルに保存します。  

In [ ]:
with open("ft_novel.txt", "w") as f:
    f.write(generated_sentences[0])